In [1]:


#pandas libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import random

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron

from sklearn.metrics import log_loss
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import Normalizer

from scipy.stats import uniform

%run "../notebooks/cargar_df.ipynb"

### Cargo los data frames con los features

In [2]:
trainX = pd.read_csv('../files_csv/train_173_features_bin_encoding')
trainX

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD,Stage
0,0,1,1,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0.52000,0.588170,5.609362e+06,0
1,1,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0.53000,0.599480,5.130851e+04,1
2,2,0,0,1,1,0,1,0,1,0,...,1,0,0,0,1,0,0.48000,0.480000,8.386560e+04,1
3,3,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53000,0.530000,7.421882e+06,0
4,4,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53000,0.530000,1.335719e+07,0
5,5,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.53000,0.530000,1.483828e+07,0
6,6,1,0,0,0,1,1,0,0,0,...,1,0,0,0,1,0,0.38000,0.380000,2.659495e+06,0
7,7,1,1,1,0,1,0,1,1,0,...,1,0,0,0,1,0,0.48000,0.480000,5.068800e+04,1
8,8,1,0,0,1,0,1,0,1,1,...,1,1,0,0,1,1,0.39034,0.471197,8.315508e+02,0
9,9,1,0,0,1,0,1,0,1,1,...,1,1,0,0,1,1,0.39034,0.471197,9.887394e+02,0


In [3]:
stages = trainX['Stage']
stages

0       0
1       1
2       1
3       0
4       0
5       0
6       0
7       1
8       0
9       0
10      0
11      1
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      1
21      1
22      1
23      1
24      1
25      0
26      0
27      1
28      1
29      1
       ..
9761    0
9762    0
9763    0
9764    1
9765    0
9766    0
9767    1
9768    1
9769    1
9770    0
9771    1
9772    1
9773    1
9774    1
9775    1
9776    1
9777    1
9778    0
9779    0
9780    0
9781    1
9782    0
9783    1
9784    1
9785    0
9786    1
9787    0
9788    1
9789    0
9790    0
Name: Stage, Length: 9791, dtype: int64

In [4]:
ids = trainX.Opportunity_ID
ids

0           0
1           1
2           2
3           3
4           4
5           5
6           6
7           7
8           8
9           9
10         10
11         11
12         12
13         13
14         14
15         15
16         16
17         17
18         18
19         19
20         20
21         21
22         22
23         23
24         24
25         25
26         26
27         27
28         28
29         29
        ...  
9761    12774
9762    12775
9763    12776
9764    12777
9765    12778
9766    12779
9767    12780
9768    12781
9769    12782
9770    12783
9771    12784
9772    12785
9773    12786
9774    12787
9775    12788
9776    12789
9777    12790
9778    12791
9779    12792
9780    12793
9781    12794
9782    12795
9783    12796
9784    12797
9785    12798
9786    12799
9787    12800
9788    12801
9789    12802
9790    12803
Name: Opportunity_ID, Length: 9791, dtype: int64

In [5]:
trainX = trainX.drop(['Opportunity_ID', 'Stage'], axis=1)

In [6]:
trainX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9791 entries, 0 to 9790
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 12.8 MB


In [7]:
testX = pd.read_csv('../files_csv/test_173_features_bin_encoding')
testX.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval_0,Bureaucratic_Code_0_Approval_1,Bureaucratic_Code_0_Approved_0,Bureaucratic_Code_0_Approved_1,bureaucratic_aprobada,amount_ids,...,Total_Amount_Currency_1,Total_Amount_Currency_2,Total_Amount_Currency_3,Total_Taxable_Amount_Currency_0,Total_Taxable_Amount_Currency_1,Total_Taxable_Amount_Currency_2,Total_Taxable_Amount_Currency_3,ASP,ASP_(converted),Total_Amount_USD
0,10689,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.306667,0.346870,137147.816349
1,10690,1,1,1,0,1,0,1,1,1,...,0,0,1,0,0,0,1,0.332000,0.375524,169716.349384
2,10691,1,1,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.425000,0.425000,21037.500000
3,10692,1,1,1,0,1,1,0,0,1,...,0,1,0,0,0,1,0,0.425000,0.425000,361517.750000
4,10693,0,0,1,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0.390000,0.390000,5752.500000


In [8]:
ids_test = testX.Opportunity_ID
ids_test

0       10689
1       10690
2       10691
3       10692
4       10693
5       10694
6       10695
7       10696
8       10697
9       10698
10      10699
11      10700
12      10701
13      10702
14      10703
15      10704
16      10705
17      10706
18      10707
19      10708
20      10709
21      10710
22      10711
23      10712
24      10713
25      10714
26      10715
27      10716
28      10717
29      10718
        ...  
1537    12339
1538    12340
1539    12341
1540    12342
1541    12343
1542    12344
1543    12345
1544    12346
1545    12347
1546    12348
1547    12349
1548    12350
1549    12351
1550    12352
1551    12353
1552    12354
1553    12355
1554    12356
1555    12357
1556    12358
1557    12359
1558    12360
1559    12361
1560    12362
1561    12363
1562    12364
1563    12365
1564    12366
1565    12367
1566    12368
Name: Opportunity_ID, Length: 1567, dtype: int64

In [9]:
testX = testX.drop('Opportunity_ID', axis=1)

In [10]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 172 entries, Pricing, Delivery_Terms_Quote_Appr to Total_Amount_USD
dtypes: float64(3), int64(169)
memory usage: 2.1 MB


# Perceptron

In [11]:
# Del archivo 'prediccion_log_regression'
# Las columnas mas influyentes para predecir con Logistic Regression

test_size = 0.33
random_state = 0

In [24]:
datos_train , datos_test, cat_train, cat_test = train_test_split(trainX,
                                                               stages,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [31]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)

In [32]:
n_iter=40
eta0 = 0.1
shuffle=False

In [33]:
#ppn con todos los features
ppn = Perceptron(n_iter_ = n_iter,
                eta0 = eta0,
                random_state = random_state,
                 shuffle = shuffle)

ppn.fit(datos_train_std,cat_train)

TypeError: __init__() got an unexpected keyword argument 'n_iter_'

In [23]:
cat_pred = ppn.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

NameError: name 'ppn' is not defined

In [25]:
final_test_data = testX[trainX.columns]

In [33]:
preds = clf.predict(final_test_data)
preds

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [32]:
prediccion = pd.DataFrame({'Opportunity_ID':ids_test, 'Target':pd.DataFrame(preds)[0]})
prediccion.head()

,Opportunity_ID,Target
0,10689,1
1,10690,1
2,10691,1
3,10692,1
4,10693,1
5,10694,1
6,10695,0
7,10696,0
8,10697,1
9,10698,1


In [34]:
prediccion.to_csv('prediccion_random_forest', index=False)